# Prepare data OrderBook 

Chỉ lấy top 5 mức giá mua, bán cao nhất trong mỗi phút cùng với khối lượng tương ứng. 
Cấu hình server có hạn, code bên dưới dùng để chuyển đổi data orderbook riêng biệt cho từng ngày

# Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import json

# Chuyển file json sang csv

In [ ]:
# Đường dẫn tới file .data
input_file = "/kaggle/input/ethusdt-orderbook-2024-01/2024-01-04_ETHUSDT_ob500.data/2024-01-04_ETHUSDT_ob500.data"
# Đường dẫn tới file .txt
output_file = "2024-01-04_ETHUSDT_ob500.txt"

# Đọc nội dung từ file .data và ghi vào file .txt
with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    data = infile.read()
    outfile.write(data)

print(f"Chuyển đổi hoàn tất! File đã được lưu với tên '{output_file}'.")

In [ ]:
# Đọc dữ liệu từ file .txt
input_file = "/kaggle/working/2024-01-04_ETHUSDT_ob500.txt"
output_file = "/kaggle/working/2024-01-04_ETHUSDT_ob500.csv"

# Mở file .txt và chuyển đổi nội dung sang CSV
with open(input_file, "r") as infile, open(output_file, "w", newline="") as outfile:
    writer = csv.writer(outfile)
    
    # Tiêu đề cột
    writer.writerow(["topic", "type", "timestamp", "symbol", "price", "size", "side", "u", "seq", "cts"])
    
    # Xử lý từng dòng JSON trong file
    for line in infile:
        record = json.loads(line.strip())  # Chuyển JSON thành dict
        topic = record["topic"]
        record_type = record["type"]
        timestamp = record["ts"]
        symbol = record["data"]["s"]
        u = record["data"]["u"]
        seq = record["data"]["seq"]
        cts = record["cts"]

        # Xử lý `bids`
        for bid in record["data"]["b"]:
            writer.writerow([topic, record_type, timestamp, symbol, bid[0], bid[1], "bid", u, seq, cts])

        # Xử lý `asks`
        for ask in record["data"]["a"]:
            writer.writerow([topic, record_type, timestamp, symbol, ask[0], ask[1], "ask", u, seq, cts])

print(f"Chuyển đổi hoàn tất! File được lưu với tên '{output_file}'.")

In [ ]:
df = pd.read_csv('/kaggle/working/2024-01-04_ETHUSDT_ob500.csv')

# Convert data 

In [ ]:
# Split data 
def find_index_at_half(df, colname):
    """splitting a DataFrame into two parts based on the cumulative distribution of a specified column's values"""
    # Count the occurrences of each value in the specified column
    value_counts = df[colname].value_counts()
    
    # Sort by index
    sorted_counts = value_counts.sort_index()
    
    # Calculate the total count
    total_count = sorted_counts.sum()
    
    # Calculate the cumulative sum
    cumulative_counts = sorted_counts.cumsum()
    
    # Calculate half of the total count
    half_total = total_count / 2
    
    # Find the index at half of the cumulative sum
    value_at_half = cumulative_counts[cumulative_counts >= half_total].index[0]
    index_at_half = df[df[colname] == value_at_half].index[0] - 1
    return index_at_half 

index_at_half = find_index_at_half(df, 'timestamp')
df1 = df.loc[:index_at_half]
df2 = df.loc[index_at_half + 1:]

df1.to_csv('/kaggle/working/2024-01-04_ETHUSDT_ob500_1.csv')
df2.to_csv('/kaggle/working/2024-01-04_ETHUSDT_ob500_2.csv')

In [ ]:
def update_delta(current, deltas):
    """
    Apply delta updates to the orderbook DataFrame sequentially by timestamp
    
    Parameters:
    -----------
    current : pd.DataFrame
    Timestamp Group of Order Book with columns: topic, type, timestamp, symbol, price, size, side, u, seq, cts
    
    Returns:
    --------
    pd.DataFrame
    Updated orderbook after processing all deltas
    """
    
    # Process each delta
    for _, delta_row in deltas.iterrows():
        # Find matching rows in the current state
        mask = (
            (current['price'] == delta_row['price']) & 
            (current['side'] == delta_row['side'])
        )
        
        # Update or add new rows
        if mask.any():
            # Update existing row
            current.loc[mask, 'size'] = delta_row['size']
        else:
            # Add new row if it doesn't exist
            current = pd.concat([
                current, 
                pd.DataFrame([delta_row])
            ])
    
    # Remove rows with zero size
    current = current[current['size'] > 0]
    
    return current.reset_index(drop=True)

In [ ]:
def convert_to_orderbook(df, max_levels=5, seq_file=1):
    
    # Group by timestamp
    timestamp_groups = df.groupby('timestamp')
    
    # Initialize results list
    orderbooks = []

    current_state = None
    if seq_file == 2:
        current_state = pd.read_csv('/kaggle/working/orderbook_2024_01_04_current_state.csv')
        updated_type_col = ['snapshot'] * len(current_state['type'])
        current_state['type'] = updated_type_col
    
    # Process each timestamp group
    for timestamp, group in timestamp_groups:

        # Separate and sort asks and bids
        asks = group[group['side'] == 'ask']
        bids = group[group['side'] == 'bid']
         
        # Prepare row for this timestamp
        if asks.empty:
            group_type = (bids['type'].values)[-1]
        else:
            group_type = (asks['type'].values)[-1]

        if group_type == 'snapshot':
            # Process snapshots first
            current_state = group
        elif group_type == 'delta':
            if current_state.empty:
                continue
            group = update_delta(current_state, group)
            current_state = group

        updated_timestamp_col = [timestamp] * len(current_state['timestamp'])
        current_state['timestamp'] = updated_timestamp_col

        # Separate and sort asks and bids
        asks = current_state[current_state['side'] == 'ask'].sort_values('price')
        bids = current_state[current_state['side'] == 'bid'].sort_values('price', ascending=False)

        row = [timestamp]
        
        # Add up to max_levels for both asks and bids
        for level in range(max_levels):
            # Ask price and size
            if level < len(asks):
                row.extend([
                    asks['price'].iloc[level],  # Ask Price
                    asks['size'].iloc[level]   # Ask Size
                ])
            else:
                row.extend([0, 0])  # Pad with zeros if no more levels
            
            # Bid price and size
            if level < len(bids):
                row.extend([
                    bids['price'].iloc[level],  # Bid Price
                    bids['size'].iloc[level]   # Bid Size
                ])
            else:
                row.extend([0, 0])  # Pad with zeros if no more levels
        
        orderbooks.append(row)

    if seq_file == 1:
        current_state.to_csv('/kaggle/working/orderbook_2024_01_04_current_state.csv')
    
    # Create column names
    columns = ['timestamp']
    for level in range(max_levels):
        columns.extend([f'ask_price_{level+1}', f'ask_size_{level+1}'])
        columns.extend([f'bid_price_{level+1}', f'bid_size_{level+1}'])
    
    # Convert to DataFrame for easy viewing/processing
    return pd.DataFrame(orderbooks, columns=columns)

In [ ]:
orderbook1 = convert_to_orderbook(df, max_levels=5, seq_file=1)

In [ ]:
orderbook1

In [ ]:
orderbook1.to_csv('orderbook_2024_01_04_ETHUSDT_1.csv')